ViT-based model using Transfer-Learning
---
#### Model: google/vit-base-patch16-224, descongelando las últimas 7 capas del encoder
#### Epochs: 30
#### Dataset: mages_3categories_balanced
#### Cambios:
- DropOut:
    - Clasificador 0.4
    - HiddenLayer 0.3
    - AttentionLayer 0.3
- Learning Rate 1e-4    

In [1]:
# Parámetros
_model = 'google/vit-base-patch16-224'
# path al checkpoint a cargar, None si no existe
_checkpoint = None
_output = 'SavedModels/ViT-base-patch16-224_F'  # path para guardar el modelo -> Revisar

_dataset = '/Users/julio/Documentos-Local/data/VinDr-Mammo/subsets/images_3categories_balanced'

# Dataset aumentado
_dataset_augmented_path = '/Users/julio/Documentos-Local/data/VinDr-Mammo/subsets/images_3categories_balanced_augmented'

# path para guardar el dataset con split
_dataset_split_path = '/Users/julio/Documentos-Local/data/VinDr-Mammo/subsets/images_3categories_balanced_augmented_split'

# Si el dataset ya está separado en train, validation y test ->  _dataset_split=_dataset_split_path. 
# Si no está separado -> _dataset_split=None.
_dataset_split = None  

_batch_size = 32
_learning_rate = 1e-4
_epochs = 30  # Epochs a entrenar el modelo

# DropOut
_dp_clasificador = 0.4
_dp_hidden_layer = 0.3
_dp_attention_layer= 0.3

num_layers_to_unfreeze = 7  # Definir el número de capas a descongelar, None eoc

In [2]:
import torch
import torch.nn as nn
import pandas as pd
from collections import Counter
from datasets import load_dataset, DatasetDict, load_from_disk,concatenate_datasets

from transformers import AutoImageProcessor, ViTForImageClassification

from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

import evaluate
from torchvision import transforms
from Utils import *

## Carga de datos

In [3]:
if _dataset_split is None:
    dataset = load_dataset(_dataset)
else:
    # Cargar el dataset previamente guardado
    dataset = load_from_disk(_dataset_split_path)

dataset

Resolving data files:   0%|          | 0/7504 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 7504
    })
})

### Extra Data Augmentation

In [4]:
# Definir transformaciones de Data Augmentation para el entrenamiento
augmentation_transforms = transforms.Compose([
    transforms.RandomRotation(10),                # Rotación aleatoria de ±10 grados
    transforms.RandomHorizontalFlip(),            # Flip horizontal aleatorio
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Cambios aleatorios de brillo, contraste, saturación y tono
])

def augment_images(batch):
    """Aplica transformaciones de Data Augmentation a las imágenes del batch."""
    augmented_images = []
    for img in batch['image']:
        # Convertir PIL Image a RGB
        img = img.convert('RGB')
        # Aplicar augmentations
        img = augmentation_transforms(img)
        augmented_images.append(img)
    batch['image'] = augmented_images
    return batch

# Aplicar Data Augmentation solo al conjunto de entrenamiento
augmented_train = dataset['train'].map(augment_images, batched=True)

# Concatenar el dataset original de entrenamiento con el aumentado
# Esto duplica el tamaño del conjunto de entrenamiento con datos aumentados
augmented_train = concatenate_datasets([dataset['train'], augmented_train])
dataset = augmented_train


### Revisión de categorías

In [5]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 15008
})

In [6]:
labels = dataset.features['label'].names
print(len(labels),labels)

label2id = {c:idx for idx,c in enumerate(labels)}
id2label = {idx:c for idx,c in enumerate(labels)}

3 ['calcificaciones', 'masas', 'no_encontrado']


#### Muestra de ejemplos

In [7]:
def show_samples(ds,rows,cols):
    samples = ds.shuffle().select(np.arange(rows*cols)) # selecting random images
    fig = plt.figure(figsize=(cols*4,rows*4))
    # plotting
    for i in range(rows*cols):
        img = samples[i]['image']
        label = samples[i]['label']
        fig.add_subplot(rows,cols,i+1)
        plt.imshow(img, cmap='gray')
        plt.title(label)
        plt.axis('off')
            
# show_samples(dataset['train'],rows=3,cols=5)

## Split Dataset

In [8]:
if _dataset_split is None:
    split_dataset = dataset.train_test_split(test_size=0.2)
    eval_dataset = split_dataset['test'].train_test_split(test_size=0.5)
    
    
    # Recombinar los splits 
    
    final_dataset = DatasetDict({
        'train': split_dataset['train'],
        'validation': eval_dataset['train'],
        'test': eval_dataset['test']
    })
    # Guardar el dataset dividido
    final_dataset.save_to_disk(_dataset_split_path)

else:
    final_dataset = dataset
final_dataset

Saving the dataset (0/1 shards):   0%|          | 0/12006 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1501 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1501 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 12006
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 1501
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 1501
    })
})

In [9]:
print('Número de imágenes por clases en cada split')
clases_split = pd.DataFrame(columns=['split', *labels])
for key in final_dataset:
    split = pd.DataFrame(final_dataset[key])
    num = split['label'].value_counts().sort_index()
    clases_split.loc[len(clases_split)] = [key, *num]
    #print(num.sort_index())
clases_split

Número de imágenes por clases en cada split


,split,calcificaciones,masas,no_encontrado
0,train,4023,3973,4010
1,validation,462,518,521
2,test,519,513,469


## Preprocesamiento de las imágenes

In [10]:
processor = AutoImageProcessor.from_pretrained(_model, use_fast=True)
processor

ViTImageProcessorFast {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTImageProcessorFast",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 224,
    "width": 224
  }
}

In [11]:
def transforms(batch):
    batch['image'] = [x.convert('RGB') for x in batch['image']]
    inputs = processor(batch['image'],return_tensors='pt')
    inputs['labels'] = batch['label']  # Las clases ya están en formato numérico
    return inputs

In [12]:
processed_dataset = final_dataset.with_transform(transforms)

### Data Collation

In [13]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

## Métricas de evaluación

In [14]:
import numpy as np
import evaluate

accuracy = evaluate.load('accuracy')
precision = evaluate.load('precision')
recall = evaluate.load('recall')
f1 = evaluate.load('f1')

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=1)

    print("Conteo de etiquetas reales:", Counter(labels))
    print("Conteo de predicciones:", Counter(predictions))
    
    precision_per_class = precision.compute(
    predictions=predictions, references=labels, average=None)
    print("Precision por clase:", precision_per_class, '\n')

    # Accuracy no requiere el parámetro average
    accuracy_score = accuracy.compute(predictions=predictions, references=labels)['accuracy']
    
    # Las demás métricas sí requieren el parámetro average para multiclase
    precision_score = precision.compute(predictions=predictions, references=labels, average='macro')['precision']
    recall_score = recall.compute(predictions=predictions, references=labels, average='macro')['recall']
    f1_score = f1.compute(predictions=predictions, references=labels, average='macro')['f1']
    
    return {
        'accuracy': accuracy_score,
        'precision': precision_score,
        'recall': recall_score,
        'f1': f1_score
    }

## Carga del modelo

In [15]:
# Clase personalizada que añade dropout antes de la capa final de clasificación
class CustomViTForImageClassification(ViTForImageClassification):
    def __init__(self, config):
        super().__init__(config)
        
        # Dropout adicional antes de la capa final
        self.additional_dropout = nn.Dropout(_dp_clasificador)  # Dropout antes del clasificador
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, pixel_values, labels=None):
        outputs = self.vit(pixel_values)  # Obtenemos la salida del modelo ViT
        
        # Usamos el primer token [CLS] de la salida
        pooled_output = outputs.last_hidden_state[:, 0]  # [CLS] está en la posición 0
        
        # Aplicamos dropout adicional antes de la clasificación
        pooled_output = self.additional_dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        loss = None
        if labels is not None:
            # Asegúrate de que las etiquetas sean tipo long (para clasificación)
            class_weights = torch.tensor([1.0, 1.0, 1.0], dtype=torch.float32)  # Aumentar peso de "sospechoso"
            loss_fn = nn.CrossEntropyLoss(weight=class_weights.to(self.device))  # Enviar los pesos al dispositivo
            loss = loss_fn(logits, labels)
        
        return (loss, logits) if loss is not None else logits

# Configuramos el modelo base con Dropout en las capas internas del ViT
model = CustomViTForImageClassification.from_pretrained(
    _model,
    num_labels = len(labels),
    id2label = id2label,
    label2id = label2id,
    hidden_dropout_prob=_dp_hidden_layer,  # Dropout en las capas internas del modelo
    attention_probs_dropout_prob=_dp_attention_layer,  # Dropout en las capas de atención
    ignore_mismatched_sizes = True
)

Some weights of CustomViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Arquitectura del modelo

In [16]:
model

CustomViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.3, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTSdpaAttention(
            (attention): ViTSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.3, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.3, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear

### Congelar todas las capas, menos el clasificador

In [17]:
for name,p in model.named_parameters():
    if not name.startswith('classifier'):
        p.requires_grad = False

In [18]:
num_params = sum([p.numel() for p in model.parameters()])
trainable_params = sum([p.numel() for p in model.parameters() if p.requires_grad])

print(f"{num_params = :,} | {trainable_params = :,}")

num_params = 85,800,963 | trainable_params = 2,307


### Descongelar capas del encoder para fine-tuning

In [19]:
# Obtener el número total de capas en el encoder
num_total_layers = len(list(model.vit.encoder.layer))  # Debería ser 24 para ViT-Large, 12 para ViT-base
print(num_total_layers)

12


In [20]:
# Si se descongelan capas
if num_layers_to_unfreeze is not None:
    # Calcular el índice a partir del cual descongelar
    unfreeze_from = num_total_layers - num_layers_to_unfreeze
    
    # Iterar sobre todas las capas del encoder
    for idx, layer in enumerate(model.vit.encoder.layer):
        if idx >= unfreeze_from:
            # Descongelar esta capa
            for param in layer.parameters():
                param.requires_grad = True
        else:
            # Congelar esta capa
            for param in layer.parameters():
                param.requires_grad = False


In [21]:
# Mostrar el número total de parámetros y los entrenables después de descongelar
num_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Después de descongelar las últimas {num_layers_to_unfreeze} capas:")
print(f"Total de parámetros: {num_params:,}")
print(f"Parámetros entrenables: {trainable_params:,}")

Después de descongelar las últimas 7 capas:
Total de parámetros: 85,800,963
Parámetros entrenables: 49,617,411


In [22]:
# Revisión de trainable por capa
for name, param in model.named_parameters():
    status = "Trainable" if param.requires_grad else "Frozen"
    #print(f"{name}: {status}")

## Training

In [23]:
training_args = TrainingArguments(
    output_dir=_output,
    per_device_train_batch_size=_batch_size,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_steps=100,
    num_train_epochs=_epochs,  # Epochs a entrenar -> Revisar
    learning_rate=_learning_rate,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to='tensorboard',
    load_best_model_at_end=True,
    metric_for_best_model="precision",  # Métrica para seleccionar el mejor modelo
    greater_is_better=True  # True si la métrica debe aumentar (ej., accuracy)
)

In [24]:
# Crear el callback de early stopping
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=10,  # Número de epochs sin mejora permitida
    early_stopping_threshold=0.0001  # Mejora mínima necesaria
)


In [25]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["validation"],
    tokenizer=processor,
    #callbacks=[early_stopping_callback]  # Añadir el callback
)

In [26]:
# Check that MPS is available
if not torch.backends.mps.is_available():
    if not torch.backends.mps.is_built():
        print("MPS not available because the current PyTorch install was not "
              "built with MPS enabled.")
    else:
        print("MPS not available because the current MacOS version is not 12.3+ "
              "and/or you do not have an MPS-enabled device on this machine.")

else:
    mps_device = torch.device("mps")
    print("MPS enabled")

MPS enabled


In [27]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.804900,0.878999,0.596269,0.597950,0.592128,0.561466
2,0.767500,0.802598,0.634244,0.640682,0.634030,0.597588
3,0.743400,0.781688,0.646236,0.649354,0.647660,0.608331
4,0.711800,0.790626,0.648235,0.652748,0.649783,0.614311
5,0.704600,0.843321,0.640240,0.649766,0.644442,0.596402
6,0.673200,0.763461,0.661559,0.658785,0.663106,0.637581
7,0.681400,0.771933,0.676882,0.677273,0.678096,0.660000
8,0.663100,0.801289,0.665556,0.670420,0.664386,0.643310
9,0.641200,0.786020,0.666223,0.670273,0.667518,0.643066
10,0.638200,0.841269,0.667555,0.682578,0.668244,0.638389


Conteo de etiquetas reales: Counter({np.int64(2): 521, np.int64(1): 518, np.int64(0): 462})
Conteo de predicciones: Counter({np.int64(2): 876, np.int64(0): 349, np.int64(1): 276})
Precision por clase: {'precision': array([0.66475645, 0.54347826, 0.58561644])} 

Conteo de etiquetas reales: Counter({np.int64(2): 521, np.int64(1): 518, np.int64(0): 462})
Conteo de predicciones: Counter({np.int64(2): 835, np.int64(0): 449, np.int64(1): 217})
Precision por clase: {'precision': array([0.66592428, 0.640553  , 0.61556886])} 

Conteo de etiquetas reales: Counter({np.int64(2): 521, np.int64(1): 518, np.int64(0): 462})
Conteo de predicciones: Counter({np.int64(2): 794, np.int64(0): 500, np.int64(1): 207})
Precision por clase: {'precision': array([0.65      , 0.65700483, 0.64105793])} 

Conteo de etiquetas reales: Counter({np.int64(2): 521, np.int64(1): 518, np.int64(0): 462})
Conteo de predicciones: Counter({np.int64(2): 766, np.int64(0): 517, np.int64(1): 218})
Precision por clase: {'precision':

KeyboardInterrupt: 

In [ ]:
trainer.save_model()

### Evaluación del modelo

In [28]:
trainer.evaluate(processed_dataset['test'])

Conteo de etiquetas reales: Counter({np.int64(0): 519, np.int64(1): 513, np.int64(2): 469})
Conteo de predicciones: Counter({np.int64(2): 737, np.int64(0): 500, np.int64(1): 264})
Precision por clase: {'precision': array([0.7       , 0.70454545, 0.61058345])} 



{'eval_loss': 1.0801012516021729,
 'eval_accuracy': 0.6568954030646236,
 'eval_precision': 0.6717096336499321,
 'eval_recall': 0.6654783893657975,
 'eval_f1': 0.6373270412348818}

## Inferencia en conjunto de test 

In [ ]:
samples = final_dataset['test']
processed_samples = samples.with_transform(transforms)
predictions = trainer.predict(processed_samples).predictions.argmax(axis=1) # labels predichas

In [ ]:
show_predictions(rows=5,cols=5, samples_=samples, predictions_=predictions, id2label_=id2label)

### Matriz de confusión

In [ ]:
confusion_matrix(samples_=samples, predictions_=predictions, class_names=labels)

## Iterar por más epochs ❌

In [ ]:
trainer.args.num_train_epochs = 30  # Para entrenar hasta la época 20
trainer.train(resume_from_checkpoint=_checkpoint)

In [ ]:
trainer.save_model()

In [ ]:
trainer.evaluate(processed_dataset['test'])

In [ ]:
confusion_matrix(samples_=samples, predictions_=predictions)